## Task 03: The Video Processing Pipeline 
**Objective:** Process a video using YOLOv8 by breaking it down into individual frames, running segmentation, and reassembling it.

#### The Pipeline Concept
Computer Vision models like YOLO typically process static images. To process video, we follow a standard 3-step pipeline:
1.  **Explode:** Split the video into individual image frames.
2.  **Inference:** Run the AI model on each frame.
3.  **Stitch:** Combine the processed frames back into a video file.

### Step 1: System Setup (FFmpeg)
We use a tool called **FFmpeg** for handling video files. 

**Installation (Windows Only):**
We used the Windows Package Manager (`winget`) to install the official build.

```powershell
# Run this in your PowerShell terminal (Administrator mode may be required)
winget install Gyan.FFmpeg
```
*Note: You must restart your terminal/VS Code after installing for the command to work.*

### Step 2: Explode Video to Frames
We need to extract frames at a specific rate. We chose **30 FPS** (Frames Per Second) to ensure smooth motion.

**Terminal Command:**
```bash
ffmpeg -i assets/test-video.mp4 -vf fps=30 task-03/frames/frame_%04d.jpg
```

**Command Breakdown:**
* `-i assets/test-video.mp4`: The input video file.
* `-vf fps=30`: **Video Filter**. Sets the extraction rate to 30 images per second.
* `frame_%04d.jpg`: The output naming pattern. `%04d` ensures files are named `frame_0001.jpg`, `frame_0002.jpg`, allowing for correct sorting later.

### Step 3: Run Inference (YOLOv8)
Now that we have a folder full of images (`task-03/frames`), we can run the YOLOv8 Segmentation model on the entire folder.

In [ ]:
from ultralytics import YOLO
import os

# 1. Load the Segmentation Model
# We use the relative path to go up one level to the 'models' folder
model = YOLO('../models/yolov8n-seg.pt') 

# 2. Run Inference
# source='frames': Looks for the folder in the current directory
# project='.': Saves results in the current directory
# name='output': Names the result folder 'output'
results = model.predict(
    source='frames', 
    save=True, 
    project='.', 
    name='output', 
    exist_ok=True,  # Overwrites if folder exists (keeps it clean)
    conf=0.5        # Confidence threshold (only show detections > 50%)
)

print("Processing complete! Frames saved to 'output' folder.")

### Step 4: Stitch Frames to Video
Finally, we use FFmpeg to combine the processed images back into a video file.

**Terminal Command:**
```bash
ffmpeg -y -framerate 30 -i output/frame_%04d.jpg -c:v libx264 -pix_fmt yuv420p final_output.mp4
```

**Command Breakdown:**
* `-y`: Automatically overwrite the file if it already exists.
* `-framerate 30`: **Critical.** Tells FFmpeg the input images should be played at 30 FPS. (Must match our extraction rate).
* `-i output/frame_%04d.jpg`: The input source pattern.
* `-c:v libx264`: The video codec (H.264 is the standard for web/compatibility).
* `-pix_fmt yuv420p`: Ensures color compatibility with all media players (Windows Media Player, QuickTime, etc.).
* `final_output.mp4`: The name of your final video.